In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
from os import listdir
from os.path import isfile, join, abspath
import pandas as pd
import xlwings as xw
import numpy as np
import quandl
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from TFANN import ANNR
import seaborn as sns
from scipy import stats
import os
%matplotlib inline 

C:\Users\clare\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# This code gets the file names of all CSVs of interests
filepath = str(input("Entire file path of CSVs: " + "\n"))
def computer(filepath):
    if filepath[2] == '\\':
        filep = filepath[2:]
        path_for_windows = filep
        path = path_for_windows.replace('\\','/')
        return path
    else:
        return filepath
p = computer(filepath)
onlyfiles = [f for f in listdir(p) if isfile(join(p, f))]
final = []
for i in onlyfiles:
    final.append(p +"/"+ i)
print(final) #C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Dat

Entire file path of CSVs: 
C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Dat
['/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_01_May_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_02_Apr_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_02_Jul_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_02_Jun_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_t

In [3]:
# Gets the currencies involved
dcc = pd.read_csv('datcommodcurr.csv',skiprows=1)#, header = None, sheet_name = 'Reference_Sheet')
dcc.index = dcc["Unnamed: 0"]
# dcc['AAP']['3/6/18']
listcurr = []
for i in dcc.columns:
    if "/" in i:
        listcurr.append(i)
listc=[]
for i in listcurr:
    listc.append(dcc[i])
dff = pd.DataFrame(listc)
dafr = dff.T
dafr["Date"] = dafr.index
dafr.index = list(range(len(dafr)))
indx = list(range(len(dafr)))
wknds = []
wkdys = []
for i in range(len(dafr)):
    try:
        if dafr.iloc[i]['AUD/CAD'] == dafr.iloc[i+1]['AUD/CAD'] and dafr.iloc[i+1]['AUD/CAD'] == dafr.iloc[i+2]['AUD/CAD']:
            wknds.append(i+1)
            wknds.append(i+2)
    except:
        continue
for i in wknds:
    if i in indx:
        indx.remove(i)
dafr
finaldf=dafr[np.in1d(np.arange(len(dafr)), indx)]
finaldf.index = finaldf["Date"]
# del finaldf['Date']
finaldf # gets df without the weekends

,AUD/CAD,AUD/CHF,AUD/EUR,AUD/GBP,AUD/JPY,AUD/NZD,AUD/USD,BTC/EUR,CAD/AUD,CAD/CHF,...,USD/NOK,USD/PLN,USD/RUB,USD/SEK,USD/SGD,USD/THB,USD/TRY,USD/UAH,USD/ZAR,Date
Date,,,,,,,,,,,,,,,,,,,,,
3/16/18,1.0100,0.7344,0.6275,0.5532,81.76,1.0682,0.7715,6935.5060,0.9891,0.7268,...,7.7131,3.4324,57.5056,8.1914,1.3173,31.23,3.9186,26.4000,11.9695,3/16/18
3/19/18,1.0090,0.7338,0.6255,0.5502,81.84,1.0649,0.7717,6828.2400,0.9905,0.7270,...,7.7219,3.4238,57.7662,8.1680,1.3168,31.18,3.9338,26.3600,12.0095,3/19/18
3/20/18,1.0041,0.7346,0.6274,0.5488,81.84,1.0689,0.7682,7280.8315,0.9956,0.7316,...,7.7470,3.4556,57.5563,8.2202,1.3187,31.18,3.9329,26.3200,11.9585,3/20/18
3/21/18,1.0013,0.7371,0.6291,0.5489,82.32,1.0740,0.7765,7207.0990,0.9981,0.7359,...,7.6955,3.4275,56.8620,8.1821,1.3122,31.15,3.9059,26.2800,11.8200,3/21/18
3/22/18,0.9949,0.7298,0.6250,0.5455,80.96,1.0670,0.7691,6988.7840,1.0042,0.7333,...,7.7489,3.4327,57.2260,8.2550,1.3167,31.30,3.9324,26.2600,11.8367,3/22/18
3/23/18,0.9921,0.7288,0.6229,0.5445,80.59,1.0631,0.7696,6975.0303,1.0073,0.7344,...,7.7543,3.4229,57.2234,8.2457,1.3152,31.18,3.9814,26.2500,11.7353,3/23/18
3/26/18,0.9945,0.7323,0.6225,0.5443,81.61,1.0608,0.7746,6339.3070,1.0046,0.7361,...,7.6810,3.3918,57.1970,8.1893,1.3085,31.12,3.9647,26.2650,11.6174,3/26/18
3/27/18,0.9889,0.7267,0.6189,0.5422,80.83,1.0570,0.7677,6444.9820,1.0104,0.7347,...,7.7175,3.3966,57.3560,8.2296,1.3096,31.21,3.9803,26.2900,11.6655,3/27/18
3/28/18,0.9896,0.7326,0.6222,0.5440,81.86,1.0624,0.7659,6417.7173,1.0095,0.7400,...,7.8621,3.4146,57.7388,8.3495,1.3114,31.27,4.0076,26.4400,11.7678,3/28/18


In [4]:
# Helper function 
def fivemax(ccsv):
    list1 = [abs(number) for number in list(ccsv['Signal'])]
    final_list = [] 
    for i in range(0, 5):  
        max1 = 0
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j]; 
        list1.remove(max1); 
        final_list.append(str(max1)) 
    return(final_list) 
csv1 = '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_09_Apr_2019.csv'
import itertools
def gettopfivedf (csv11):
    csv = pd.read_csv(csv11)
    for1 = csv[0:10]
    for2 = csv[10:20]
    for3 = csv[20:30]
    for4 = csv[30:40]
    for5 = csv[40:50]
    for6 = csv[50:60]
    for2.index= list(range(len(for2)))
    for3.index= list(range(len(for3)))
    for4.index= list(range(len(for4)))
    for5.index= list(range(len(for5)))
    for6.index= list(range(len(for6)))
    dlist = [for1,for2,for3,for4,for5,for6]
    futdf=[]
    for y in dlist:
        for r in fivemax(y):
            for i in range(len(y)):
                if r in str(y["Signal"][i]):
                    futdf.append([y['Ticker'][i],y['Time Horizon'][i],y['Predictability'][i],y['Signal'][i]])
    gf = pd.DataFrame(futdf, columns = ["Currency", "Time Horizon", "Predictability", "Signal"])
    dateee=(csv11[-15:].replace("_","/").replace(".csv","").replace("May","05").replace("Mar","03").replace("Apr","04").replace("Jun","06").replace("Jul","07").replace("Aug","08"))
    datetimeobject = datetime.strptime(dateee,'%d/%m/%Y') # to change from dd/mm/yyyy to a datetime object
    final_date = datetimeobject.strftime('%d/%m/%Y')# to change to mm/dd/yyyy
    L = list(itertools.repeat(final_date, len(gf)))
    gf["Date"] = L
    return(gf)
listofdfs=[]
for u in final:
    fin = gettopfivedf(u)
    listofdfs.append(fin)
df_top_5 = pd.concat(listofdfs)
df_top_5.index = list(range(len(df_top_5)))
df_top_5 = df_top_5.applymap(str)
df_top_5

,Currency,Time Horizon,Predictability,Signal,Date
0,RUB/GBP,1d,0.15,-10.775369760345272,01/05/2019
1,AUD/GBP,1d,0.16,-8.27926040675232,01/05/2019
2,GBP/AUD,1d,0.14,7.712282818873827,01/05/2019
3,USD/IDR,1d,0.12,7.2774579093211225,01/05/2019
4,EUR/RUB,1d,0.13,6.607806002460362,01/05/2019
5,RUB/GBP,2d,0.18,-8.979474800287727,01/05/2019
6,AUD/GBP,2d,0.18,-7.819301495266081,01/05/2019
7,GBP/AUD,2d,0.16,7.306373196827834,01/05/2019
8,USD/IDR,2d,0.14,7.2774579093211225,01/05/2019
9,EUR/RUB,2d,0.15,6.19481812730659,01/05/2019


In [5]:
DATA=[]
specific =[]
for u in final:
# u = 'C:/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_01_May_2019.csv' 
    actual = u[-15:]
    datee = (u[-15:].replace("_","/").replace(".csv","").replace("2019","19").replace("May","5").replace("Mar","3").replace("Apr","4").replace("Jun","6").replace("Jul","7").replace("Aug","8")
)
    date = datee.replace("0","")
    dateee=(u[-15:].replace("_","/").replace(".csv","").replace("May","05").replace("Mar","03").replace("Apr","04").replace("Jun","06").replace("Jul","07").replace("Aug","08")
)
    df = pd.read_csv(u)
    for i in range (len(df)):
        for curr in listcurr:
            if df['Ticker'][i] == curr:
                DATA.append([date, df['Ticker'][i],df['Time Horizon'][i],df['Predictability'][i],df['Signal'][i]])
                specific.append([dateee, df['Ticker'][i],df['Time Horizon'][i],df['Predictability'][i],df['Signal'][i]])         
e = pd.DataFrame(specific)
e.columns=(["Date","Currency","Time Horizon","Predictability","Signal"])
e.index = e["Date"]
t = pd.DataFrame(DATA)
t.columns=(["Date","Currency","Time Horizon","Predictability","Signal"])
t.index = t["Date"]
e # dd/mm/yyyy df

,Date,Currency,Time Horizon,Predictability,Signal
Date,,,,,
01/05/2019,01/05/2019,GBP/AUD,1d,0.14,7.712283
01/05/2019,01/05/2019,USD/IDR,1d,0.12,7.277458
01/05/2019,01/05/2019,EUR/RUB,1d,0.13,6.607806
01/05/2019,01/05/2019,GBP/USD,1d,0.10,5.623930
01/05/2019,01/05/2019,GBP/EUR,1d,0.10,3.956948
01/05/2019,01/05/2019,JPY/CNY,1d,0.15,3.920768
01/05/2019,01/05/2019,CHF/JPY,1d,0.10,-3.497643
01/05/2019,01/05/2019,EUR/GBP,1d,0.11,-3.961699
01/05/2019,01/05/2019,AUD/GBP,1d,0.16,-8.279260


In [6]:
# helper function
import datetime
def addbusdays(from_date, add_days):
    business_days_to_add = add_days
    current_date = from_date
    while business_days_to_add > 0:
        current_date += datetime.timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    return current_date
def results (pair,Hor): 
    # this function creates a table and chart for returns and signal
    cp = pair
    TimeHorizon = Hor
    dayreturn = int([char for char in TimeHorizon][0])
    currpair = []
    for r in range(len(e)):
        if e["Currency"][r] == cp and e["Time Horizon"][r] == TimeHorizon:
            # datetimeobject = datetime.strptime(e["Date"][r],'%d/%m/%Y') # to change from dd/mm/yyyy to a datetime object
            currpair.append([e["Date"][r], e["Signal"][r]])# to change to mm/dd/yyyy
    g = pd.DataFrame(currpair)
    g.columns = ["Date", "Signal"]
    # g['Date'] =pd.to_datetime(g.Date)
    # k = g.sort_values('Date') # sort by date
    g.index = g["Date"]
    forcst = []
    retdate = []
    of =[]
    for i in g["Date"]:
        q = datetime.datetime.strptime(i,'%d/%m/%Y') # remove time
        of.append(q.strftime("%m/%d/%Y"))
        forcst.append(q.strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
        retdate.append(addbusdays(q, dayreturn).strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
    g["old format"] = of
    g["Forecast Date"]=forcst
    g["End of return"] = retdate
    dattes = list(finaldf.index)
    newdlist=[]
    for i in range(len(g)):
        if g["Forecast Date"][i] not in dattes:
            newf = (datetime.datetime.strptime(g["old format"][i],"%m/%d/%Y"))-timedelta(days=2)
            newdlist.append(newf.strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
        else:
            newdlist.append(g["Forecast Date"][i])
    g["Start of return"]=newdlist
    
    returns=[]
    for i in range(len(g)):
        try:
            returns.append(((dcc[cp][g["End of return"][i]] - dcc[cp][g["Start of return"][i]])/dcc[cp][g["Start of return"][i]])*100)
        except:
            returns.append(None)
    g["Returns"]= returns
    alif = []
    alll = []
    for i in range(len(g)):
        if (g["Signal"][i] > 0 and g["Returns"][i] > 0) or (g["Signal"][i] < 0 and g["Returns"][i] < 0):
            alll.append(abs(g["Returns"][i]))
        else:
            if g["Returns"][i] < 0:
                alif.append(g["Returns"][i])
            else:
                alif.append(-1*g["Returns"][i])
    alif = [x for x in alif if str(x) != 'nan']
    alll = [x for x in alll if str(x) != 'nan'] 

    if len(alif) > 0:
        average_losses= sum(alif)/len(alif)
    else:
        average_losses= 0
    if len(alll) >0:
        average_gains= sum(alll)/len(alll)
    else:
        average_gains= 0
    average_returns = average_gains + average_losses #Average returns just by following signal per currency pair
    hit_ratio = (len(alll)/len(g))*100
    untouched = []
    for i in g["Returns"]:
        untouched.append(i)
    untouched = [x for x in untouched if str(x) != 'nan']
    real_returns = sum(untouched)/len(untouched) # returns generated by the market (Long position)
    if average_returns > real_returns:
        beaten = "Success"
    else: 
        beaten = "Failure"
    return([cp, average_returns, real_returns, hit_ratio,beaten,len(g), TimeHorizon])
results("AUD/GBP","4d")

['AUD/GBP',
 0.13004171304060597,
 0.2224920418055275,
 58.620689655172406,
 'Failure',
 29,
 '4d']

In [7]:
# this part churns out results for long and short strategy
# If this was a day trader of currency pairs following our forecasts,
# he would long a pair or short a pair if signal is positive or negative respectively
# the returns are under the header "IKF Long and Short strategy
# the benchmark is calculated by taking the average of the market performance for each day the trader trades

results1=[]
for i in listcurr:
    try:
        results1.append(results (i , "1d"))# toggle the time horizon to see results
        results1.append(results (i , "2d"))
        results1.append(results (i , "3d"))
        results1.append(results (i , "4d"))
        results1.append(results (i , "5d"))
        results1.append(results (i , "6d"))
    except:
        continue
        
dfresults = pd.DataFrame(results1,columns=["Currency Pair","I Know First Long and Short Strategy", "Benchmark Returns (Long Strategy): ","I Know First Hit Ratio: ","Success?","Appearances in Top 10","Time Horizon"])
dfresults

,Currency Pair,I Know First Long and Short Strategy,Benchmark Returns (Long Strategy):,I Know First Hit Ratio:,Success?,Appearances in Top 10,Time Horizon
0,AUD/GBP,0.101214,0.026343,43.750000,Success,32,1d
1,AUD/GBP,0.024313,0.097136,56.666667,Failure,30,2d
2,AUD/GBP,0.215535,0.241605,56.666667,Failure,30,3d
3,AUD/GBP,0.130042,0.222492,58.620690,Failure,29,4d
4,AUD/GBP,-0.116726,0.235690,61.290323,Failure,31,5d
5,AUD/GBP,0.182524,0.207287,50.000000,Failure,30,6d
6,AUD/JPY,0.187164,-0.081077,100.000000,Success,2,1d
7,AUD/JPY,0.050371,0.050371,100.000000,Failure,1,2d
8,AUD/JPY,-0.064693,0.225747,85.714286,Failure,7,3d
9,AUD/JPY,-0.736369,0.841702,66.666667,Failure,3,4d


In [8]:
def addbusdays(from_date, add_days):
    business_days_to_add = add_days
    current_date = from_date
    while business_days_to_add > 0:
        current_date += timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    return current_date
def resultstop5 (pair,Hor): 
# this function creates a table and chart for returns and signal
    cp = pair
    TimeHorizon = Hor
    dayreturn = int([char for char in TimeHorizon][0])
    currpair = []
    # for specific time horizon and specific stock
    for r in range(len(df_top_5)):
        if df_top_5["Currency"][r] == cp and df_top_5["Time Horizon"][r] == TimeHorizon:
            # datetimeobject = datetime.strptime(e["Date"][r],'%d/%m/%Y') # to change from dd/mm/yyyy to a datetime object
            currpair.append([df_top_5["Date"][r], df_top_5["Signal"][r]])# to change to mm/dd/yyyy

    g = pd.DataFrame(currpair)
    g.columns = ["Date", "Signal"]
    g.index = g["Date"]
    forcst = []
    retdate = []
    of =[]
    for i in g["Date"]:
        q = datetime.datetime.strptime(i,'%d/%m/%Y') # remove time
        of.append(q.strftime("%m/%d/%Y"))
        forcst.append(q.strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
        retdate.append(addbusdays(q, dayreturn).strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
    g["old format"] = of
    g["Forecast Date"]=forcst
    g["End of return"] = retdate

    dattes = list(finaldf.index)
    newdlist=[]
    for i in range(len(g)):
        if g["Forecast Date"][i] not in dattes:
            newf = (datetime.datetime.strptime(g["old format"][i],"%m/%d/%Y"))-timedelta(days=2)
            newdlist.append(newf.strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
        else:
            newdlist.append(g["Forecast Date"][i])
    g["Start of return"]=newdlist
    returns=[]
    for i in range(len(g)):
        try:
            returns.append(((dcc[cp][g["End of return"][i]] - dcc[cp][g["Start of return"][i]])/dcc[cp][g["Start of return"][i]])*100)
        except:
            continue
    g["Returns"]= returns
    alif = []
    alll = []
    for i in range(len(g)):
        if (float(g["Signal"][i]) > 0 and g["Returns"][i] > 0) or (float(g["Signal"][i]) < 0 and g["Returns"][i] < 0):
            alll.append(abs(g["Returns"][i]))
        else:
            if g["Returns"][i] < 0:
                alif.append(g["Returns"][i])
            else:
                alif.append(-1*g["Returns"][i])
    alif = [x for x in alif if str(x) != 'nan']
    alll = [x for x in alll if str(x) != 'nan'] 
    po
    if len(alif) > 0:
        average_losses= sum(alif)/len(alif)
    else:
        average_losses= 0
    if len(alll) >0:
        average_gains= sum(alll)/len(alll)
    else:
        average_gains= 0
    average_returns = average_gains + average_losses #Average returns just by following signal per currency pair
    hit_ratio = (len(alll)/len(g))*100
    untouched = []
    for i in g["Returns"]:
        untouched.append(i)
    untouched = [x for x in untouched if str(x) != 'nan']
    real_returns = sum(untouched)/len(untouched) # returns generated by the market (Long position)
    if average_returns > real_returns:
        beaten = "Success"
    else: 
        beaten = "Failure"
    return([cp, average_returns, real_returns, hit_ratio,beaten,len(g), TimeHorizon])

In [9]:
results5top=[]
for i in listcurr:
    try:
        results5top.append(resultstop5 (i , "1d"))# toggle the time horizon to see results
        results5top.append(resultstop5 (i , "2d"))
        results5top.append(resultstop5 (i , "3d"))
        results5top.append(resultstop5 (i , "4d"))
        results5top.append(resultstop5 (i , "5d"))
        results5top.append(resultstop5 (i , "6d"))
    except:
        continue
        
dfresults1 = pd.DataFrame(results5top,columns=["Currency Pair","I Know First Long and Short Strategy", "Benchmark Returns (Long Strategy): ","I Know First Hit Ratio: ","Success?","Appearances in Top 5", "TimeHorizon"])
dfresults1

,Currency Pair,I Know First Long and Short Strategy,Benchmark Returns (Long Strategy):,I Know First Hit Ratio:,Success?,Appearances in Top 5,TimeHorizon
0,AUD/GBP,0.172115,0.018097,50.000000,Success,12,1d
1,AUD/GBP,0.257058,-0.107059,44.444444,Success,9,2d
2,AUD/GBP,0.031557,-0.001053,37.500000,Success,8,3d
3,AUD/GBP,-0.187961,0.044052,40.000000,Failure,5,4d
4,AUD/GBP,-0.321539,0.219029,20.000000,Failure,5,5d
5,AUD/GBP,-0.436080,0.436080,0.000000,Failure,3,6d
6,AUD/USD,0.075870,0.156812,45.454545,Failure,11,1d
7,AUD/USD,0.184663,0.239896,20.000000,Failure,5,2d
8,AUD/USD,0.062437,-0.088616,50.000000,Success,4,3d
9,AUD/USD,0.489484,-0.227733,33.333333,Success,3,4d


In [10]:
export_excel = dfresults.to_excel (r'C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Analysis\IKF Analysis.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

print (dfresults)

    Currency Pair  I Know First Long and Short Strategy  \
0         AUD/GBP                              0.101214   
1         AUD/GBP                              0.024313   
2         AUD/GBP                              0.215535   
3         AUD/GBP                              0.130042   
4         AUD/GBP                             -0.116726   
5         AUD/GBP                              0.182524   
6         AUD/JPY                              0.187164   
7         AUD/JPY                              0.050371   
8         AUD/JPY                             -0.064693   
9         AUD/JPY                             -0.736369   
10        AUD/JPY                              0.941399   
11        AUD/JPY                              0.878871   
12        AUD/USD                             -0.035272   
13        AUD/USD                              0.197481   
14        AUD/USD                              0.073755   
15        AUD/USD                             -0.014512 

In [11]:

export_excel = dfresults1.to_excel (r'C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Analysis\IKF Analysis1.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

print (dfresults1)

    Currency Pair  I Know First Long and Short Strategy  \
0         AUD/GBP                              0.172115   
1         AUD/GBP                              0.257058   
2         AUD/GBP                              0.031557   
3         AUD/GBP                             -0.187961   
4         AUD/GBP                             -0.321539   
5         AUD/GBP                             -0.436080   
6         AUD/USD                              0.075870   
7         AUD/USD                              0.184663   
8         AUD/USD                              0.062437   
9         AUD/USD                              0.489484   
10        AUD/USD                              0.213327   
11        AUD/USD                              0.567736   
12        BTC/EUR                             10.391700   
13        BTC/EUR                              8.098489   
14        BTC/EUR                             12.333853   
15        BTC/EUR                             12.508237 